In [2]:
from GoogleNews import GoogleNews
import json 

In [ ]:
from flask import Flask
from flask import jsonify



import spacy
from collections import Counter
from string import punctuation

# nlp = spacy.load("en_core_web_lg")


# import en_core_web_lgnlp 
# nlp = en_core_web_lg.load()

# from GoogleNews import GoogleNews
# import json
app = Flask(__name__)


import urllib.request
from newspaper import Article
from bs4 import BeautifulSoup as Soup

# def top_sentence(text, limit):
#     keyword = []
#     pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
#     doc = nlp(text.lower())
#     for token in doc:
#         if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
#             continue
#         if(token.pos_ in pos_tag):
#             keyword.append(token.text)
    
#     freq_word = Counter(keyword)
#     max_freq = Counter(keyword).most_common(1)[0][1]
#     for w in freq_word:
#         freq_word[w] = (freq_word[w]/max_freq)
        
#     sent_strength={}
#     for sent in doc.sents:
#         for word in sent:
#             if word.text in freq_word.keys():
#                 if sent in sent_strength.keys():
#                     sent_strength[sent]+=freq_word[word.text]
#                 else:
#                     sent_strength[sent]=freq_word[word.text]
    
#     summary = []
    
#     sorted_x = sorted(sent_strength.items(), key=lambda kv: kv[1], reverse=True)
    
#     counter = 0
#     for i in range(len(sorted_x)):
#         summary.append(str(sorted_x[i][0]).capitalize())

#         counter += 1
#         if(counter >= limit):
#             break
            
#     return ' '.join(summary)


class GoogleNews:

    def __init__(self,lang="en",period="",start="",end=""):
        self.__texts = []
        self.__links = []
        self.__results = []
        self.user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:64.0) Gecko/20100101 Firefox/64.0'
        self.headers = {'User-Agent': self.user_agent}
        self.__lang = lang
        self.__period = period
        self.__start = start
        self.__end = end

    def setlang(self, lang):
        self.__lang = lang

    def setperiod(self, period):
        self.__period = period

    def setTimeRange(self, start, end):
        self.__start = start
        self.__end = end
        
    def search(self, key):
        """
        Searches for a term in google news and retrieves the first page into __results.
        
        Parameters:
        key = the search term
        """
        self.__key = "+".join(key.split(" "))
        self.getpage()

    def getpage(self, page=1):
        """
        Retrieves a specific page from google news into __results.
        Parameter:
        page = number of the page to be retrieved 
        """
        try:
            if self.__start != "" and self.__end != "":
                self.url = "https://www.google.com/search?q={}&lr=lang_{}&tbs=lr:lang_1{},cdr:1,cd_min:{},cd_max:{}&tbm=nws&start={}".format(self.__key,self.__lang,self.__lang,self.__start,self.__end,(10 * (page - 1)))
            elif self.__period != "":
                self.url = "https://www.google.com/search?q={}&lr=lang_{}&tbs=lr:lang_1{},qdr:{},&tbm=nws&start={}".format(self.__key,self.__lang,self.__lang,self.__period,(10 * (page - 1))) 
            else:
                self.url = "https://www.google.com/search?q={}&lr=lang_{}&tbs=lr:lang_1{}&tbm=nws&start={}".format(self.__key,self.__lang,self.__lang,(10 * (page - 1))) 
        except AttributeError:
            raise AttributeError("You need to run a search() before using getpage().")
        try:
            self.req = urllib.request.Request(self.url, headers=self.headers)
            self.response = urllib.request.urlopen(self.req)
            self.page = self.response.read()
            self.content = Soup(self.page, "html.parser")
            result = self.content.find_all("div", id="search")[0].find_all("g-card")
            for item in result:
                try:
#                     print(item)
                    tmp_text = item.findAll("g-img")[1].parent.findNext('div').text.replace("\n","")
#                     tmp_text = item.find("div", {"role" : "heading"}).text.replace("\n","")
                except Exception:
                    tmp_text = ''
                try:
                    tmp_link = item.find("a").get("href")
                except Exception:
                    tmp_link = ''
                try:
                    tmp_media = item.findAll("g-img")[1].parent.text
                except Exception:
                    tmp_media = ''
                try:
                    tmp_date = item.find("div", {"role" : "heading"}).next_sibling.findNext('div').findNext('div').text
                except Exception:
                    tmp_date = ''
                try:
                    tmp_desc = item.find("div", {"role" : "heading"}).next_sibling.findNext('div').text.replace("\n","")
                except Exception:
                    tmp_desc = ''
                try:
                    article = Article(tmp_link)
                    article.download()
                    article.parse()
#                     article.nlp()
                    tmp_img = article.top_image
#                     tmp_img = item.findAll("g-img")[0].find("img").get('src')
#                     print(article.text)
                except Exception:
                    tmp_img = ''
                self.__texts.append(tmp_text)
                self.__links.append(tmp_link)
                self.__results.append({'title': tmp_text, 'media': tmp_media,'date': tmp_date,'desc': tmp_desc, 'link': tmp_link,'img': tmp_img})
            self.response.close()
        except Exception:
            pass

    def get_news(self, deamplify=False):
        self.url = 'https://news.google.com/?hl={}'.format(self.__lang)
        try:
            self.req = urllib.request.Request(self.url, headers=self.headers)
            self.response = urllib.request.urlopen(self.req)
            self.page = self.response.read()
            self.content = Soup(self.page, "html.parser")
            self.content = self.content.find("h2").parent.parent.parent
            result = self.content.findChildren("div", recursive=False)
            section = None
            for item in result:
                try:
                    try:
                        section = item.find("h2").find("a").text
                    except Exception as sec_e:
                        pass
                    title = item.find("h3").text
                    if deamplify:
                        try:
                            link = item.find("article").get("jslog").split('2:')[1].split(';')[0]
                        except Exception as deamp_e:
                            print(deamp_e)
                            link = 'news.google.com/' + item.find("h3").find("a").get("href")
                    else:
                        link = item.find("h3").find("a").get("href")
                    self.__texts.append(title)
                    self.__links.append(link)
                    try:
                        datetime = item.find("time").get("datetime")
                    except:
                        datetime = None
                    try:
                        time = item.find("time").text
                    except:
                        time = None
                    try:
                        site = item.find("time").parent.find("a").text
                    except:
                        site = None
                    try:
                        article = Article(tmp_link)
                        article.download()
                        article.parse()
#                     article.nlp()
                        img = article.top_image
#                         img = item.find("img").get("src")
                    except:
                        img = None
                    desc = None
                    if link.startswith('https://www.youtube.com/watch?v='):
                        desc = 'video'

                    self.__results.append(
                        {'section': section,
                         'title': title,
                         'datetime': datetime,
                         'time': time,
                         'site': site,
                         'desc': desc,
                         'link': link,
                         'media': None,
                         'img': img})
                except Exception as big_e:
                    pass
            self.response.close()
        except Exception as e:
            print(e)
            pass

    def result(self):
        """Returns the __results."""
        return self.__results
    

    def gettext(self):
        """Returns only the __texts of the __results."""
        return self.__texts

    def get__links(self):
        """Returns only the __links of the __results."""
        return self.__links

    def clear(self):
        self.__texts = []
        self.__links = []
        self.__results = []
@app.route("/")
def hello():
    googlenews = GoogleNews()
    googlenews = GoogleNews(lang='te')
    googlenews.search('covid')
    # result = googlenews
    # for i in range(1,20):
    #     googlenews.getpage(i)
    # googlenews.getpage()

    result = googlenews.result()
    print(len(result))
    return jsonify(result)


if __name__ == '__main__':
    app.run(debug=False)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
C:\ProgramData\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1206: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
127.0.0.1 - - [16/Jul/2020 22:14:30] "GET / HTTP/1.1" 200 -


16


In [4]:
print(*result)

{'title': 'Uttar Pradesh comes up with new strategy to tackle Covid-19, offices and markets to remain closed on...', 'media': 'Hindustan Times', 'date': '2 hours ago', 'desc': 'The state has already imposed a 55-hour mini lockdown till Monday, where except for essential and some other services, everything else is ...', 'link': 'https://www.hindustantimes.com/india-news/uttar-pradesh-comes-up-with-new-strategy-to-tackle-covid-19-offices-and-markets-to-remain-closed-on-weekends/story-U246rEkvNu64gORxUPimJK.html', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='} {'title': '', 'media': 'Times Now', 'date': '', 'desc': '', 'link': 'https://www.timesnownews.com/india/article/aishwarya-rai-bachchan-daughter-aaradhya-test-covid-19-positive/620275', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='} {'title': '', 'media': 'The Indian Express', 'date': '', 'desc': '', 'link': 'https://indianexpress.com/article/

In [5]:
from newspaper import Article

In [21]:
article = Article('https://www.timesnownews.com/health/article/airborne-transmission-of-covid1-9-who-issues-new-guidelines-here-s-what-you-can-do-to-stay-safe/619268')

In [32]:
article.download()
article.parse()
article.nlp()

In [34]:
article.authors

['Updated Jul']

In [35]:
article.publish_date

In [33]:
article.keywords

['droplet',
 'settings',
 'transmission',
 'issues',
 'stay',
 'virus',
 'heres',
 'guidelines',
 'spread',
 'covid19',
 'infected',
 'safe',
 'airborne',
 'respiratory',
 'health']

In [36]:
print(article.text)

Airborne transmission of COVID-19: WHO issues new guidelines; here’s how you can stay safe from coronavirus | Photo Credit: iStock Images

Key Highlights The World Health Organization has acknowledged that there's possibility of coronavirus being spread in the air under certain conditons The UN body releases new guidelines on COVID-19 transmission, including new scientific evidence available on the spread of the SARS-CoV-2 Here's all you need to know about airborne transmission and what you can do to stay safe from the deadly virus

New Delhi: The World Health Organization (WHO) on Thursday issued new guidelines on the transmission of the novel coronavirus acknowledging some reports of airborne spread of the SARS-CoV-2 virus that causes COVID-19. In an updated scientific brief on COVID-19 transmission, the UN health agency said airborne spread of coronavirus can occur in health care settings where specific medical procedures generate very small droplets - aerosols. However, WHO said th

In [37]:
print(article.summary)

In an updated scientific brief on COVID-19 transmission, the UN health agency said airborne spread of coronavirus can occur in health care settings where specific medical procedures generate very small droplets - aerosols.
However, WHO said that more research is urgently required to elucidate the importance of different transmission routes of the virus.
In this article, let us tell you how airborne transmission is different from droplet transmission and what you can do to stay safe from respiratory infections such as COVID-19.
What’s the difference between airborne and droplet transmission?
Airborne transmission is different from droplet transmission.


In [31]:
article.top_image

'https://imgk.timesnownews.com/story/covid-transmission.gif?tr=w-560,h-292,fo-top'